Imports

In [33]:
import warnings
warnings.filterwarnings('ignore')

Extract Atrophy Within Region of Interest

___

## Neuroimaging File Extraction

Here's a breakdown of the inputs:

### 0. Base Directorty (CSF)
- **base_directory**: Absolute path to the base directory containing CSF files.

### 1. Cerebrospinal Fluid (CSF)
- **glob_name_pattern**: File pattern to search for CSF files.

### 2. Grey Matter
- **glob_name_pattern**: File pattern to search for grey matter files.

### 3. White Matter
- **glob_name_pattern**: File pattern to search for white matter files.

---

**Instructions**: Please fill out the `base_directory` and `glob_name_pattern` fields for each segment. This will ensure that the extraction process can locate and identify the appropriate neuroimaging files for further analysis.

---

Input Directory
- This is the BIDS-style directory which was created by Notebook 02

In [1]:
# Shared Base Directory
base_directory = r'/Users/cu135/Dropbox (Partners HealthCare)/studies/atrophy_seeds_2023/shared_analysis/niftis_for_elmira/wmaps/sbm/6mm_w_map_bids_final'

This is the Glob-style path to the subfolder containing niftis of interest
- For example, from the base_directory, */*/tissue_segment_z_scores will look for all subjects, all session folders within subjects, and then check the tissue_segment_z_scores folder. 


In [2]:
shared_glob_path = r'*/vol'

This is the list of tissue types to use from within each terminal folder. 
- tissue_to_import = ['cerebrospinal_fluid', 'white_matter', 'grey_matter']
- **Leave as unchanged if you are unsure**

In [3]:

tissue_to_import = ['vol_MNI152_6.nii.gz']

In [4]:
import os
from calvin_utils.file_utils.import_matrices import import_matrices_from_folder
def import_dataframes_by_tissue(base_directory, shared_glob_path, tissue_to_import):
    """
    Imports dataframes based on tissue types from specified directories and glob name patterns.
    
    Parameters:
    - base_directory (str): The base directory where the data resides.
    - shared_glob_path (str): The shared directory path for all tissues.
    - tissue_to_import (list): List of tissues to be imported.
    
    Returns:
    - dict: A dictionary containing dataframes for each specified tissue.
    """

    segments_dict = {}
    for tissue in tissue_to_import:
        glob_path = os.path.join(shared_glob_path, ('*'+tissue+'*'))
        segments_dict[tissue] = glob_path

    dataframes_dict = {}
    nan_handler = {'nan': 0, 'posinf':20, 'neginf':-20}
    for k, v in segments_dict.items():
        dataframes_dict[k] = import_matrices_from_folder(connectivity_path=base_directory, file_pattern=v, convert_nan_to_num=nan_handler)
        print(f'Imported data {k} data with {dataframes_dict[k].shape[0]} voxels and {dataframes_dict[k].shape[1]} patients')
        print(f'Example filename per subject: {dataframes_dict[k].columns[0]}')
        print('\n--------------------------------\n')

    return dataframes_dict


In [5]:
imported_dataframes_by_tissue = import_dataframes_by_tissue(base_directory, shared_glob_path, tissue_to_import)

I will search:  /Users/cu135/Dropbox (Partners HealthCare)/studies/atrophy_seeds_2023/shared_analysis/niftis_for_elmira/wmaps/sbm/6mm_w_map_bids_final/*/vol/*vol_MNI152_6.nii.gz*
Imported data vol_MNI152_6.nii.gz data with 902629 voxels and 49 patients
Example filename per subject: sub-0015_vol_MNI152_6.nii.gz

--------------------------------



**Extract Subject ID From File Names**
Using the example filenames that have been printed above, please define a general string:
1) Preceding the subject ID.
2) Proceeding the subject ID. 

In [6]:
from calvin_utils.file_utils.dataframe_utilities import extract_and_rename_subject_id
def rename_dataframe_subjects(dataframes_dict, preceding_id, proceeding_id):
    """
    Renames the subjects in the provided dataframes based on the split commands.

    Parameters:
    - dataframes_dict (dict): A dictionary containing dataframes with subjects to be renamed.
    - preceding_id (str): The delimiter for taking the part after the split.
    - proceeding_id (str): The delimiter for taking the part before the split.

    Returns:
    - dict: A dictionary containing dataframes with subjects renamed.
    """

    split_command_dict = {preceding_id: 1, proceeding_id: 0}
    
    for k, v in dataframes_dict.items():
        dataframes_dict[k] = extract_and_rename_subject_id(dataframe=dataframes_dict[k], split_command_dict=split_command_dict)
        print('Dataframe: ', k)
        display(dataframes_dict[k])
        print('------------- \n')

    return dataframes_dict


This Should Often Be Left Default

In [9]:
preceding_id = 'sub-'
proceeding_id = '_vol'

In [10]:
thresholded_atrophy_df_dict = rename_dataframe_subjects(imported_dataframes_by_tissue, preceding_id, proceeding_id)

Dataframe:  vol_MNI152_6.nii.gz


,0015,0012,0024,0023,0048,0046,0041,0022,0025,0013,...,0010,0032,0035,0003,0004,0050,0005,0002,0034,0033
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902624,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902626,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902627,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


------------- 



# Extract Region of Interest Atrophy

Import Region of Interest Masks

In [11]:
folder_to_import_from = '/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/Software/VBM/rois'
shared_glob_pattern = '*'

In [12]:
from calvin_utils.file_utils.import_matrices import import_matrices_from_folder
region_of_interest_df = import_matrices_from_folder(connectivity_path=folder_to_import_from, file_pattern=shared_glob_pattern)
region_of_interest_df

I will search:  /Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/Software/VBM/rois/*


,mni_Cerebellum.nii.gz,mni_152_cortex_mask_2mm.nii,subcortex_mask_2mm.nii.gz,mni_Temporal_Lobe.nii.gz,MNI152_T1_2mm_brain_mask.nii,hippocampus_2mm.nii,mni_Occipital_Lobe.nii.gz,mni_Frontal_Lobe.nii.gz,mni_Parietal_Lobe.nii.gz,mni_Insular_Lobe.nii.gz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
902624,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
902625,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
902626,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
902627,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0


Extract Damage Scores Per Region of Interest

In [13]:
import pandas as pd
import numpy as np
def calculate_damage_scores(thresholded_atrophy_df_dict, region_of_interest_df, count_voxels=True):
    """
    Calculates damage scores for each region of interest based on thresholded atrophy data.
    
    Parameters:
    - thresholded_atrophy_df_dict (dict): Dictionary containing dataframes with thresholded atrophy data.
    - region_of_interest_df (DataFrame): DataFrame containing regions of interest.
    
    Returns:
    - dict: A dictionary containing damage scores for each region of interest for each patient.
    """
    
    damage_df_dict = {}
    
    for k, v in thresholded_atrophy_df_dict.items():
        
        # Initialize the Dataframe
        damage_df_dict[k] = pd.DataFrame(index=v.columns, columns=region_of_interest_df.columns)
        
        # Iterate Over Each Region of Interest
        for matrix in damage_df_dict[k].columns:
            # Extract Damage Score for Each Patient
            for subject in damage_df_dict[k].index:
                # Mask the subject dataframe to the matrix at hand
                intersection = v[subject].where(region_of_interest_df[matrix] > 0, 0)
                
                # Use multiplication to zero values outside ROI
                weighted_overlap = intersection * region_of_interest_df[matrix]
                
                # Assess overall damage score & atrophied voxels within ROI
                damage_score = weighted_overlap.sum()
                num_voxels = np.count_nonzero(weighted_overlap)
                
                # Assign values to DF 
                damage_df_dict[k].loc[subject, matrix] = damage_score
                damage_df_dict[k].loc[subject, 'num_atrophied_voxels_'+matrix] = num_voxels
                
        print('Dataframe: ', k)
        display(damage_df_dict[k])
        print('------------- \n')
    
    return damage_df_dict


In [14]:
damage_df_dict = calculate_damage_scores(thresholded_atrophy_df_dict, region_of_interest_df)

Dataframe:  vol_MNI152_6.nii.gz


,mni_Cerebellum.nii.gz,mni_152_cortex_mask_2mm.nii,subcortex_mask_2mm.nii.gz,mni_Temporal_Lobe.nii.gz,MNI152_T1_2mm_brain_mask.nii,hippocampus_2mm.nii,mni_Occipital_Lobe.nii.gz,mni_Frontal_Lobe.nii.gz,mni_Parietal_Lobe.nii.gz,mni_Insular_Lobe.nii.gz,num_atrophied_voxels_mni_Cerebellum.nii.gz,num_atrophied_voxels_mni_152_cortex_mask_2mm.nii,num_atrophied_voxels_subcortex_mask_2mm.nii.gz,num_atrophied_voxels_mni_Temporal_Lobe.nii.gz,num_atrophied_voxels_MNI152_T1_2mm_brain_mask.nii,num_atrophied_voxels_hippocampus_2mm.nii,num_atrophied_voxels_mni_Occipital_Lobe.nii.gz,num_atrophied_voxels_mni_Frontal_Lobe.nii.gz,num_atrophied_voxels_mni_Parietal_Lobe.nii.gz,num_atrophied_voxels_mni_Insular_Lobe.nii.gz
0015,0.0,337.678413,111.265302,50.085092,448.943714,-2.537025,-25.764459,215.370344,130.356244,37.508821,0.0,1762.0,461.0,373.0,2223.0,336.0,269.0,767.0,688.0,43.0
0012,0.0,-1090.033178,-316.871208,-258.806499,-1406.904386,-2.36312,-121.189812,-396.090259,-579.409977,-3.999716,0.0,1762.0,461.0,373.0,2223.0,336.0,269.0,767.0,688.0,43.0
0024,0.0,302.414046,-58.898846,25.413262,243.5152,0.834293,-15.010491,101.44591,264.803031,-35.723487,0.0,1762.0,461.0,373.0,2223.0,336.0,269.0,767.0,688.0,43.0
0023,0.0,-88.749877,25.136697,26.024708,-63.61318,1.161686,32.463195,-151.348906,-11.734673,1.161349,0.0,1762.0,461.0,373.0,2223.0,336.0,269.0,767.0,688.0,43.0
0048,0.0,-987.563204,-335.951901,-229.817093,-1323.515105,-2.631654,-155.322179,-423.034555,-402.07719,-29.586612,0.0,1762.0,461.0,373.0,2223.0,336.0,269.0,767.0,688.0,43.0
0046,0.0,-1137.964294,-375.548425,-185.134397,-1513.51272,-0.492643,-246.188158,-458.769339,-554.532053,8.041267,0.0,1762.0,461.0,373.0,2223.0,336.0,269.0,767.0,688.0,43.0
0041,0.0,-1746.046055,-537.320698,-310.256549,-2283.366754,-1.224376,-233.708284,-821.397637,-739.199705,-57.401833,0.0,1762.0,461.0,373.0,2223.0,336.0,269.0,767.0,688.0,43.0
0022,0.0,-123.021784,49.795665,-121.698214,-73.226118,-4.060455,75.111857,-64.852155,23.50776,2.28163,0.0,1762.0,461.0,373.0,2223.0,336.0,269.0,767.0,688.0,43.0
0025,0.0,-1339.648023,-473.255102,-431.910144,-1812.903125,1.019621,-355.463492,-499.211425,-414.41515,-23.232023,0.0,1762.0,461.0,373.0,2223.0,336.0,269.0,767.0,688.0,43.0
0013,0.0,-241.451994,-56.806728,2.719641,-298.258722,-0.059935,-98.343158,72.77967,-224.365607,-15.219996,0.0,1762.0,461.0,373.0,2223.0,336.0,269.0,767.0,688.0,43.0


------------- 



Extract Total Atrophy Volume

In [15]:
import numpy as np
from nimlab import datasets as nimds
def calculate_total_atrophy_voxels(thresholded_atrophy_df_dict, damage_df_dict):
    """
    Calculates the total number of atrophy voxels within a mask and updates the damage_df_dict with this information.
    
    Parameters:
    - thresholded_atrophy_df_dict (dict): Dictionary containing dataframes with thresholded atrophy data.
    - damage_df_dict (dict): Dictionary containing dataframes to which the 'Total Atrophy Voxels' column will be added.
    
    Returns:
    - dict: The updated damage_df_dict with a new column 'Total Atrophy Voxels'.
    """
    
    # Get the mask and brain indices
    mni_mask = nimds.get_img("mni_icbm152")
    mask_data = mni_mask.get_fdata().flatten()
    brain_indices = np.where(mask_data > 0)[0]
    
    for k, v in thresholded_atrophy_df_dict.items():
        # initialize the column
        damage_df_dict[k]['Total Atrophy Voxels'] = np.nan
        for patient_id in v.columns:
            # Filter the dataframe using brain indices
            filtered_df = v[patient_id].iloc[brain_indices]
            damage_df_dict[k].loc[patient_id, 'Total Atrophy Voxels'] = np.count_nonzero(filtered_df)
        
        print('Dataframe: ', k)
        display(damage_df_dict[k])
        print('------------- \n')
    
    return damage_df_dict


In [16]:
damage_df_dict = calculate_total_atrophy_voxels(thresholded_atrophy_df_dict, damage_df_dict)

Dataframe:  vol_MNI152_6.nii.gz


,mni_Cerebellum.nii.gz,mni_152_cortex_mask_2mm.nii,subcortex_mask_2mm.nii.gz,mni_Temporal_Lobe.nii.gz,MNI152_T1_2mm_brain_mask.nii,hippocampus_2mm.nii,mni_Occipital_Lobe.nii.gz,mni_Frontal_Lobe.nii.gz,mni_Parietal_Lobe.nii.gz,mni_Insular_Lobe.nii.gz,...,num_atrophied_voxels_mni_152_cortex_mask_2mm.nii,num_atrophied_voxels_subcortex_mask_2mm.nii.gz,num_atrophied_voxels_mni_Temporal_Lobe.nii.gz,num_atrophied_voxels_MNI152_T1_2mm_brain_mask.nii,num_atrophied_voxels_hippocampus_2mm.nii,num_atrophied_voxels_mni_Occipital_Lobe.nii.gz,num_atrophied_voxels_mni_Frontal_Lobe.nii.gz,num_atrophied_voxels_mni_Parietal_Lobe.nii.gz,num_atrophied_voxels_mni_Insular_Lobe.nii.gz,Total Atrophy Voxels
0015,0.0,337.678413,111.265302,50.085092,448.943714,-2.537025,-25.764459,215.370344,130.356244,37.508821,...,1762.0,461.0,373.0,2223.0,336.0,269.0,767.0,688.0,43.0,2223.0
0012,0.0,-1090.033178,-316.871208,-258.806499,-1406.904386,-2.36312,-121.189812,-396.090259,-579.409977,-3.999716,...,1762.0,461.0,373.0,2223.0,336.0,269.0,767.0,688.0,43.0,2223.0
0024,0.0,302.414046,-58.898846,25.413262,243.5152,0.834293,-15.010491,101.44591,264.803031,-35.723487,...,1762.0,461.0,373.0,2223.0,336.0,269.0,767.0,688.0,43.0,2223.0
0023,0.0,-88.749877,25.136697,26.024708,-63.61318,1.161686,32.463195,-151.348906,-11.734673,1.161349,...,1762.0,461.0,373.0,2223.0,336.0,269.0,767.0,688.0,43.0,2223.0
0048,0.0,-987.563204,-335.951901,-229.817093,-1323.515105,-2.631654,-155.322179,-423.034555,-402.07719,-29.586612,...,1762.0,461.0,373.0,2223.0,336.0,269.0,767.0,688.0,43.0,2223.0
0046,0.0,-1137.964294,-375.548425,-185.134397,-1513.51272,-0.492643,-246.188158,-458.769339,-554.532053,8.041267,...,1762.0,461.0,373.0,2223.0,336.0,269.0,767.0,688.0,43.0,2223.0
0041,0.0,-1746.046055,-537.320698,-310.256549,-2283.366754,-1.224376,-233.708284,-821.397637,-739.199705,-57.401833,...,1762.0,461.0,373.0,2223.0,336.0,269.0,767.0,688.0,43.0,2223.0
0022,0.0,-123.021784,49.795665,-121.698214,-73.226118,-4.060455,75.111857,-64.852155,23.50776,2.28163,...,1762.0,461.0,373.0,2223.0,336.0,269.0,767.0,688.0,43.0,2223.0
0025,0.0,-1339.648023,-473.255102,-431.910144,-1812.903125,1.019621,-355.463492,-499.211425,-414.41515,-23.232023,...,1762.0,461.0,373.0,2223.0,336.0,269.0,767.0,688.0,43.0,2223.0
0013,0.0,-241.451994,-56.806728,2.719641,-298.258722,-0.059935,-98.343158,72.77967,-224.365607,-15.219996,...,1762.0,461.0,373.0,2223.0,336.0,269.0,767.0,688.0,43.0,2223.0


------------- 



Organize the Subjects

In [17]:
def sort_dataframes_by_index(damage_df_dict):
    """
    Attempts to convert the index of each dataframe in damage_df_dict to integers 
    and then sorts the dataframe by its index in ascending order.
    
    Parameters:
    - damage_df_dict (dict): Dictionary containing dataframes to be sorted.
    
    Returns:
    - dict: The sorted damage_df_dict.
    """
    
    sorted_df_dict = {}
    
    for k, df in damage_df_dict.items():
        try:
            # Convert index to integers
            df.index = df.index.astype(int)
            
            # Sort dataframe by index
            sorted_df = df.sort_index(ascending=True)
            sorted_df_dict[k] = sorted_df
            
            # Display the results
            print('Dataframe: ', k)
            display(sorted_df)
            print('------------- \n')
            
        except ValueError:
            # If conversion to integer fails, just use the original dataframe
            sorted_df_dict[k] = df

    return sorted_df_dict


In [18]:
sorted_damage_df_dict = sort_dataframes_by_index(damage_df_dict)

Dataframe:  vol_MNI152_6.nii.gz


,mni_Cerebellum.nii.gz,mni_152_cortex_mask_2mm.nii,subcortex_mask_2mm.nii.gz,mni_Temporal_Lobe.nii.gz,MNI152_T1_2mm_brain_mask.nii,hippocampus_2mm.nii,mni_Occipital_Lobe.nii.gz,mni_Frontal_Lobe.nii.gz,mni_Parietal_Lobe.nii.gz,mni_Insular_Lobe.nii.gz,...,num_atrophied_voxels_mni_152_cortex_mask_2mm.nii,num_atrophied_voxels_subcortex_mask_2mm.nii.gz,num_atrophied_voxels_mni_Temporal_Lobe.nii.gz,num_atrophied_voxels_MNI152_T1_2mm_brain_mask.nii,num_atrophied_voxels_hippocampus_2mm.nii,num_atrophied_voxels_mni_Occipital_Lobe.nii.gz,num_atrophied_voxels_mni_Frontal_Lobe.nii.gz,num_atrophied_voxels_mni_Parietal_Lobe.nii.gz,num_atrophied_voxels_mni_Insular_Lobe.nii.gz,Total Atrophy Voxels
1,0.0,-470.165311,-147.966774,-147.211766,-618.132084,-3.503051,-135.954946,-178.567699,-175.969548,37.029264,...,1762.0,461.0,373.0,2223.0,336.0,269.0,767.0,688.0,43.0,2223.0
2,0.0,499.502671,180.360016,15.554683,679.862688,1.446747,67.572269,424.169344,113.693553,19.553744,...,1762.0,461.0,373.0,2223.0,336.0,269.0,767.0,688.0,43.0,2223.0
3,0.0,-311.339795,-26.49369,-222.185755,-337.833486,-6.235943,-17.902266,131.443135,-235.607711,18.091693,...,1762.0,461.0,373.0,2223.0,336.0,269.0,767.0,688.0,43.0,2223.0
4,0.0,-1206.712467,-479.205127,-11.164522,-1685.917594,0.28619,-281.317216,-645.805327,-699.602622,14.569066,...,1762.0,461.0,373.0,2223.0,336.0,269.0,767.0,688.0,43.0,2223.0
5,0.0,-768.592739,-180.828752,-176.023407,-949.421492,-1.561098,-104.818014,-353.205797,-277.162059,-13.492183,...,1762.0,461.0,373.0,2223.0,336.0,269.0,767.0,688.0,43.0,2223.0
6,0.0,-306.384856,-155.695333,8.735008,-462.080189,1.974489,-171.499284,-68.442241,-211.748623,-1.447041,...,1762.0,461.0,373.0,2223.0,336.0,269.0,767.0,688.0,43.0,2223.0
7,0.0,0.056273,-78.549703,-73.33861,-78.49343,-0.891255,-59.357756,-20.96662,135.124016,-3.975682,...,1762.0,461.0,373.0,2223.0,336.0,269.0,767.0,688.0,43.0,2223.0
8,0.0,-1333.045727,-447.035213,-352.912577,-1780.08094,-3.33312,-187.749455,-589.921503,-485.340246,-48.853907,...,1762.0,461.0,373.0,2223.0,336.0,269.0,767.0,688.0,43.0,2223.0
9,0.0,-1674.587863,-518.590359,-324.575642,-2193.178222,0.441953,-288.203404,-613.74916,-855.398589,-24.688448,...,1762.0,461.0,373.0,2223.0,336.0,269.0,767.0,688.0,43.0,2223.0
10,0.0,368.90724,93.641216,79.501586,462.548457,0.353687,89.121086,166.081291,146.909141,-2.551473,...,1762.0,461.0,373.0,2223.0,336.0,269.0,767.0,688.0,43.0,2223.0


------------- 



Save the Results

In [19]:
import os
from calvin_utils.nifti_utils.generate_nifti import view_and_save_nifti
from tqdm import tqdm

def save_csv_to_bids(dataframes_dict, bids_base_dir, analysis='atrophy_results', ses=None, dry_run=True):
    """
    Saves csv to a BIDS directory structure.
    
    Parameters:
    - dataframes_dict (dict): Dictionary containing dataframes with NIFTI data.
    - bids_base_dir (str): The base directory where the BIDS structure starts.
    - ses (str, optional): Session identifier. If None, defaults to '01'.
    
    Note:
    This function assumes a predefined BIDS directory structure and saves the CSV 
    accordingly.
    """
    
    for key, value in tqdm(dataframes_dict.items()):            
        # Define BIDS Directory Architecture
        sub_no = 'all'
        if ses is None:
            ses_no = '01'
        else:
            ses_no = ses
        
        # Define and Initialize the Save Directory
        out_dir = os.path.join(bids_base_dir, 'neuroimaging_analyses', f'ses-{ses_no}', f'sub-{sub_no}', analysis)
        os.makedirs(out_dir, exist_ok=True)
        
        # Save Image to BIDS Directory
        if dry_run:
            print(out_dir+f'/{key}.csv')
        else:
            value.to_csv(out_dir+f'/{key}.csv')
            print('Saved to: ', out_dir+f'/{key}.csv')

In [20]:
analysis='6mm_wmap_atrophy'

In [21]:
save_csv_to_bids(sorted_damage_df_dict, bids_base_dir=base_directory, analysis=analysis, ses=None, dry_run=False)

100%|██████████| 1/1 [00:00<00:00, 96.29it/s]

Saved to:  /Users/cu135/Dropbox (Partners HealthCare)/studies/atrophy_seeds_2023/shared_analysis/niftis_for_elmira/wmaps/sbm/6mm_w_map_bids_final/neuroimaging_analyses/ses-01/sub-all/6mm_wmap_atrophy/vol_MNI152_6.nii.gz.csv


All done. Enjoy your analyses. 

--Calvin 